In [1]:
!pip install torch
!pip install torchmetrics
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 9.0 MB/s eta 0:00:00a 0:00:01


# Imports
Packages required for experiment

In [2]:
# General use for DL
import torch

# Used to split data
from sklearn.model_selection import KFold

# To loader dataset
from torch.utils.data import DataLoader

# Data Measurements
from torchmetrics import JaccardIndex
from torchmetrics import Dice

# Dataset import
from dataset import BUIDSegmentationDataset

# Stopping Rule
When this function returns a value less than 0.1 % = 0.001 then we know to stop training

In [3]:
# Finds percent change between previous and current loss
# and if it is less than threshold return false
def stopping_rule(L_k, L_k1, threshold):
    return abs(L_k - L_k1) / L_k > threshold

In [4]:
# Calculates the moving average
def moving_avg(alpha, L_MA, L_k):
    return alpha * L_MA + (1-alpha) * L_k

# Training Function


In [5]:
def train_model(model, loss_fn, device, train_loader, optimizer):
    # Initalize loss
    average_loss = 0
    # Train on dataset
    model.train()
    for batch_idx, (X,y) in enumerate(train_loader):
        # Get batch
        image, mask = X.to(device), y.to(device)
        # Get results
        output = model(image)
        # Compute loss
        loss = loss_fn(output, mask)
        average_loss += loss.item()
        # Optimize model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Return average loss
    return average_loss / len(train_loader)
        
        

# Testing Function

In [6]:
def test_model(model, device, test_loader, jaccard, dice):
    # Initalize average jaccard and dice
    average_jaccard = 0
    average_dice = 0
    # Test the model
    model.eval()
    for batch_idx, (X,y) in enumerate(test_loader):
        # Get batch
        image, mask = X.to(device), y.to(device)
        # Get results
        output = model(image)
        average_jaccard += jaccard(torch.where(output > 0.5, 1, 0),torch.where(mask > 0.50, 1, 0)).item()
        average_dice += dice(torch.where(output > 0.5, 1, 0),torch.where(mask > 0.50, 1, 0)).item()
    # Get average of dice and jaccard scores
    average_jaccard /= len(test_loader)
    average_dice /= len(test_loader)

    # Return values
    return average_jaccard, average_dice

# Data Processing
Importing the dataset into the Juypter Notebook enviroment for use

In [7]:
# Path to images and mask
root_dir = './BUID'
# U-Net we are using takes 3 x 256 x 256 images
size = 256
# Importing the dataset
dataset = BUIDSegmentationDataset(root_dir, size)


Loading preliminary variables for the experiment:
- Jaccard score
- Dice score
- Loss fuction
- Device
- KFold
- Batch size
- Stopping threshold

In [8]:
## Preliminary variables ##

# Specifies whether to train on GPU or CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Loss for training
loss_fn = torch.nn.BCELoss()

# Measurements
jaccard = JaccardIndex(task='multiclass', num_classes = 2, average = 'micro').to(device)
dice = Dice(num_classes = 2, average = 'micro').to(device)

# K-Fold object for splitting dataset, randomizes batches (shuffle = True)
splits = 5
kf = KFold(splits, shuffle=True)

# Batch Size
BATCH_SIZE = 16

# Stopping threshold
threshold = 0.001

# Speeds up training
num_workers = 8

# Alpha for EMA
alpha = 0.9

# Base Model and Metrics
First we must create a baseline result to compare to. Using K-fold validation we will train 5 seperate models:
1. Using K-fold validation we will attain 5 splits of 20% of the dataset {A,B,C,D,E}
2. We stash away one of the splits, ex. A, for testing and train on the remaining 4 splits until we reach the stopping rule
3. Test the train model on the test splits and save the Jaccard and Dice scores
4. Repeat the previous steps but stash away a different split, ex. B,C,D, or E, until we have tested on all splits.
5. Take the average of all the Dice and Jaccard scores to achieve the baseline results

To optimize the model we will begin our learning rate at 0.01, use Cosine Annealing LR to decay the learning rate, and the Adam method of optimization.

After the experiment is done we will save the indices of each fold and the baseline metrics to use in semi-automatic segmentation

In [9]:
# Dictionary to save each fold
fold_name = ['A','B','C','D','E'] # To save indicies of each fold
fold_dict = {}
# Initialize Jaccard and Dice
average_jaccard = 0
average_dice = 0
average_iterations = 0

# Split dataset and determine baseline jaccard and dice
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    
    ## Initialize the datasets ##
    
    # Get train loader for fold
    train_loader = DataLoader(
        dataset=dataset,
        batch_size=BATCH_SIZE,
        sampler=torch.utils.data.SubsetRandomSampler(train_idx),
        num_workers = num_workers
    )
    # Get test loader for fold
    test_loader = DataLoader(
        dataset=dataset,
        batch_size=BATCH_SIZE,
        sampler=torch.utils.data.SubsetRandomSampler(test_idx),
        num_workers = num_workers
    )
    
    # Save indices in dictionary for future experiments
    fold_dict.update({fold_name[fold] : [train_idx, test_idx]})
    
    ## Initialize the model ##

    # Loading an untrained model to GPU/CPU
    model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
        in_channels=3, out_channels=1, init_features=64, pretrained=False, trust_repo=True).to(device)
    # We will begin our learning rate at 0.01 
    lr = 0.01
    # Optimizer for model
    optimizer = torch.optim.Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,25)
    
    ## Initialize the training ##

    L_MA = 1 # Moving average of loss, initialize to 0 so no division by zero error
    L_k = 0 # Current loss

    # To determine in threshold is too low
    counter = 0
    # Train model until stopping rule is reached
    while(stopping_rule(L_MA, L_k, threshold) or counter < 10):
        # Train model and compute loss
        L_k = train_model(model, loss_fn, device, train_loader, optimizer)

        # Initialization
        if(L_MA == 0):
            L_MA = L_k

        # Find EMA of losses
        L_MA = moving_avg(alpha, L_MA, L_k)
        counter += 1
    # To determine in threshold is too low
    print(f"Iterations:{counter} | Loss: {L_k}")
    # Test model on remaining splits
    jaccard_i, dice_i = test_model(model, device, test_loader, jaccard, dice)
    average_jaccard += jaccard_i
    average_dice += dice_i
    average_iterations += counter

# Take average of jaccard and dice of all 5 models
average_jaccard /= 5    
average_dice /= 5
average_iterations /= 5

# Print results
print(f"Baseline Jaccard: {average_jaccard}")
print(f"Baseline Dice: {average_dice}")

# Save the indicies and baseline scores into a state dict for future use
name = 'baseline.pt'
state = {
    'fold_dict' : fold_dict,
    'baseline_jaccard' : average_jaccard,
    'baseline_dice' : average_dice,
    'baseline_iterations' : average_iterations
}
torch.save(state, f = name)

Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip


Iterations:446 | Loss: 0.003184073324052569


Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


Iterations:248 | Loss: 0.003909795533101528


Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


Iterations:190 | Loss: 0.004509060267502299


Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


Iterations:384 | Loss: 0.0035497269963320247


Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


Iterations:436 | Loss: 0.0033825317529054023
Baseline Jaccard: 0.9213233041763307
Baseline Dice: 0.9587881147861481
